In [1]:
%pip install optlang
from optlang import Model, Variable, Constraint, Objective
import pandas as pd

Note: you may need to restart the kernel to use updated packages.


## A Model of Production and Transportation

In [2]:
#creating indexes
origins = ["GARY", "CLEV", "PITT"]
dests = ["FRA", "DET", "LAN", "WIN", "STL", "FRE", "LAF"]
prods = ["bands", "coils", "plate"]

#storing data in dictionaries
availability = {
    "GARY": 20, 
    "CLEV": 15, 
    "PITT": 20
}
demand = {
    "FRA": [300, 500, 100],
    "DET": [300, 750, 100],
    "LAN": [100, 400, 0],
    "WIN": [75, 250, 50],
    "STL": [650, 950, 200],
    "FRE": [225, 850, 100],
    "LAF": [250, 500, 250]
}
rate = {
    "GARY": [200, 140, 160], 
    "CLEV": [190, 130, 160], 
    "PITT": [230, 160, 170]
}
make_cost = {
    "GARY": [180, 170, 180],
    "CLEV": [190, 170, 185],
    "PITT": [190, 180, 185]
}
trans_cost_bands = {
    "FRA": [30, 22, 19],
    "DET": [10, 7, 11],
    "LAN": [8, 10, 12],
    "WIN": [10, 7, 10],
    "STL": [11, 21, 25],
    "FRE": [71, 82, 83],
    "LAF": [6, 13, 15]
}
trans_cost_coils = {
    "FRA": [39, 27, 24],
    "DET": [14, 9, 14],
    "LAN": [11, 12, 17],
    "WIN": [14, 9, 13],
    "STL": [16, 26, 28],
    "FRE": [82, 95, 99],
    "LAF": [8, 17, 20]
}
trans_cost_plate = {
    "FRA": [41, 29, 26],
    "DET": [15, 9, 14],
    "LAN": [12, 13, 17],
    "WIN": [16, 9, 13],
    "STL": [17, 28, 31],
    "FRE": [86, 99, 104],
    "LAF": [8, 18, 20]
}

In [3]:
#creating data frames for data
df_demand = pd.DataFrame(demand, index=prods)
df_rate = pd.DataFrame(rate, index=prods)
df_make_cost = pd.DataFrame(make_cost, index=prods)
df_trans_cost_bands = pd.DataFrame(trans_cost_bands, index=origins)
df_trans_cost_coils = pd.DataFrame(trans_cost_coils, index=origins)
df_trans_cost_plate = pd.DataFrame(trans_cost_plate, index=origins)
df_cost = pd.concat((df_trans_cost_bands,df_trans_cost_coils,df_trans_cost_plate),keys=prods,axis=0)

print("cost 3D dataframe:")
df_cost

cost 3D dataframe:


FRA  DET  LAN  WIN  STL  FRE  LAF
bands GARY   30   10    8   10   11   71    6
      CLEV   22    7   10    7   21   82   13
      PITT   19   11   12   10   25   83   15
coils GARY   39   14   11   14   16   82    8
      CLEV   27    9   12    9   26   95   17
      PITT   24   14   17   13   28   99   20
plate GARY   41   15   12   16   17   86    8
      CLEV   29    9   13    9   28   99   18
      PITT   26   14   17   13   31  104   20

In [4]:
print("rate dataframe:")
df_rate

rate dataframe:


,GARY,CLEV,PITT
bands,200,190,230
coils,140,130,160
plate,160,160,170


In [5]:
print(f"demand dataframe:")
df_demand

demand dataframe:


,FRA,DET,LAN,WIN,STL,FRE,LAF
bands,300,300,100,75,650,225,250
coils,500,750,400,250,950,850,500
plate,100,100,0,50,200,100,250


In [6]:
#creating variables in dictionaries
#products manufactured in origin location
prod_made = {}
for origin in origins:
    prod_made[origin] = [Variable(name=f"{prod}_made_in_{origin}", lb = 0, type='integer') for prod in prods]

#products shipped from gary to destination
trans_gary = {}
for dest in dests:
    trans_gary[dest] = [Variable(name=f"{prod}_shipped_from_gary_to_{dest}", lb = 0, type='integer') for prod in prods]

#products shipped from clev to destination
trans_clev = {}
for dest in dests:
    trans_clev[dest] = [Variable(name=f"{prod}_shipped_from_clev_to_{dest}", lb = 0, type='integer') for prod in prods]

#products shipped from pitt to destination
trans_pitt = {}
for dest in dests:
    trans_pitt[dest] = [Variable(name=f"{prod}_shipped_from_pitt_to_{dest}", lb = 0, type='integer') for prod in prods]

#converting dictionaries to dataframes
df_prod_made = pd.DataFrame(prod_made, index=prods)
df_trans_gary = pd.DataFrame(trans_gary, index=prods)
df_trans_clev = pd.DataFrame(trans_clev, index=prods)
df_trans_pitt = pd.DataFrame(trans_pitt, index=prods)
#3D dataframe for specific products shipped from specific origin shipped to specific destination
df_trans = pd.concat([df_trans_gary, df_trans_clev, df_trans_pitt], keys=origins)

In [7]:
#making constraints
constraints = []

#time constraint
constraints.append(list(Constraint(
    expression=sum((1/df_rate.loc[prod][origin])*df_prod_made.loc[prod][origin] for prod in prods), 
    name=f"{origin}_time_constraint",
    ub = availability[origin] 
) for origin in origins))
    
#supply constraint  
constraints.append(list(Constraint(
    expression=sum(df_trans.loc[origin][dest][prod] for dest in dests) - df_prod_made.loc[prod][origin], 
    name=f"supply_constraint_for_{prod}_from_{origin}",
    lb = 0, ub = 0
) for origin in origins for prod in prods))

#demand constraint       
constraints.append(list(Constraint(
     expression=sum(df_trans.loc[origin][dest][prod] for origin in origins) - df_demand.loc[prod][dest], 
     name=f"demand_constraint_for_{prod}_{dest}",
     lb = 0, ub = 0
) for dest in dests for prod in prods))

In [8]:
#creating objective function 
objective = sum(df_make_cost.loc[prod][origin] * df_prod_made.loc[prod][origin] for origin in origins for prod in prods) + sum(df_cost.loc[prod][dest][origin] * df_trans.loc[origin][dest][prod] for origin in origins for dest in dests for prod in prods)

#assembling model and finding optimal solution
model = Model(name="production_and_transportation")
model.add(constraints)
model.objective = Objective(expression= objective, direction="min")

#optimizing and printing output
print(f"Model: {model.name}")
status = model.optimize()
print("Status:", status)
print(f"Objective value: ${model.objective.value}\n")
print("non-zero variables: ")
for var in model.variables:
    if var.primal != 0:
        print(var.name, f": {var.primal} units")
print("")
print("zero variables: ")
for var in model.variables:
    if var.primal == 0:
        print(var.name, f": {var.primal} units")

Model: production_and_transportation
Status: optimal
Objective value: $1392175.0

non-zero variables: 
plate_made_in_GARY : 300.0 units
bands_made_in_GARY : 1125.0 units
coils_made_in_GARY : 1750.0 units
coils_made_in_CLEV : 1950.0 units
bands_made_in_PITT : 775.0 units
plate_made_in_PITT : 500.0 units
coils_made_in_PITT : 500.0 units
bands_shipped_from_gary_to_LAF : 250.0 units
bands_shipped_from_gary_to_STL : 650.0 units
bands_shipped_from_gary_to_FRE : 225.0 units
coils_shipped_from_gary_to_FRE : 850.0 units
coils_shipped_from_gary_to_STL : 900.0 units
plate_shipped_from_gary_to_STL : 200.0 units
plate_shipped_from_gary_to_FRE : 100.0 units
coils_shipped_from_clev_to_STL : 50.0 units
coils_shipped_from_clev_to_WIN : 250.0 units
coils_shipped_from_clev_to_LAF : 500.0 units
coils_shipped_from_clev_to_DET : 750.0 units
coils_shipped_from_clev_to_LAN : 400.0 units
bands_shipped_from_pitt_to_FRA : 300.0 units
bands_shipped_from_pitt_to_DET : 300.0 units
bands_shipped_from_pitt_to_WIN : 7

As we can see, the output is identical to the AMPL textbook, hence, we assume the implementation is correct.